In [0]:
name: Dynamic Databricks Notebook Deploy

on:
  push:
    branches:
      - main

jobs:
  deploy:
    runs-on: ubuntu-latest

    steps:
    - name: Checkout Repository
      uses: actions/checkout@v3

    - name: Install jq, curl, and Python
      run: sudo apt-get update && sudo apt-get install -y jq curl python3

    - name: Export multiple notebooks (DBC format)
      run: |
        ORIGIN_HOST=${{ secrets.DATABRICKS_ORIGIN_HOST }}
        ORIGIN_TOKEN=${{ secrets.DATABRICKS_ORIGIN_TOKEN }}
        NOTEBOOK_BASE="/Workspace/Users/developer1@anadeyanirahotmail.onmicrosoft.com/sales"
        NOTEBOOKS=("Orquestador","INGEST_SALES","SILVER","GOLDEN","Conexion-AzureSQL","Untitled Notebook 2025-11-13 18:46:19")  # Agrega más notebooks según lo necesites

        mkdir -p notebooks_to_deploy

        for nb in "${NOTEBOOKS[@]}"; do
          # URL encode el nombre del notebook
          encoded_nb=$(python3 -c "import urllib.parse, sys; print(urllib.parse.quote(sys.argv[1]))" "$nb")
          
          echo "Exportando $nb (codificado como $encoded_nb)..."
          curl -s -X GET \
            -H "Authorization: Bearer $ORIGIN_TOKEN" \
            "$ORIGIN_HOST/api/2.0/workspace/export?path=$NOTEBOOK_BASE/$encoded_nb&format=DBC&direct_download=true" \
            --output "notebooks_to_deploy/$encoded_nb.dbc"

          # Verificar si el archivo está vacío
          if [ ! -s "notebooks_to_deploy/$encoded_nb.dbc" ]; then
            echo "Advertencia: El archivo exportado $encoded_nb.dbc está vacío."
          fi
        done

    - name: Deploy notebooks to Destination Works
